In [ ]:
import LinearAlgebra

In [ ]:
import Random

In [ ]:
import BenchmarkTools

In [ ]:
import Profile

In [ ]:
import StaticArrays
# details on staticarrays here https://m3g.github.io/JuliaNotes.jl/stable/immutable/

# test scoring function
formulate as a minimization problem

In [ ]:
function Score(Answer)
    return(ceil(Int64, (LinearAlgebra.norm(Answer .- 500))^2))
end

# overall implementation
this function runs the particle simulations

In [ ]:
function TestAlgorithm(dims = 3, nparticles = 10, maxiter = 500)
    # generate the starting points for the algorithm
    # store these in a matrix, where each column is a particle, and each row is a dimension in the search space
    particles = [StaticArrays.SVector{dims, Int64}(rand(1:1000, dims)) for i in 1:nparticles]
    velocities = [StaticArrays.SVector{dims, Int64}(rand(-3:3, dims)) for i in 1:nparticles]
    direction = [StaticArrays.SVector{dims, Int64}(zeros(Int64, dims)) for i in 1:nparticles]
    scores = zeros(Int64, nparticles)

    #scores = StaticArrays.@MVector zeros(Int64, nparticles)
    
    # create a set to store the points that we have visited, to avoid looking at the same points repeatedly
    visited = Set{StaticArrays.SVector{dims, Int64}}()
    
    # give a size hint for the set of visited points, hopefully preventing it from growing repeatedly
    sizehint!(visited, nparticles * maxiter)
    
    iter = 1
    
    best_particle = zeros(Int64, dims)
    global_optimum = Inf
        
    wiggled = 0
    # loop until convergence
    # while iter < maxiter
    while iter <= maxiter
        # "wiggle" each particle position if it's already been visited
        idx = findall(x -> x in visited, particles)
        
        # for each of these, try wiggling
        for part in idx
            wiggled += 1
            newpart = particles[part]
            tries = 1
            while (tries < 40) & (newpart in visited)
                dist = Int(floor(tries/4 + 1))
                newpart = particles[part] .+ (rand(1:dist, dims) .* rand([-1, 1], dims))
                tries += 1
            end
            particles[part] = newpart
        end
        
        # compute the current scores of each particle
        map!(x -> Score(x), scores, particles)
            
        # find the current best particle
        best_score = findmin(scores)
        
        # if this score is better than the current best score, update
        if best_score[1] < global_optimum[1]
            global_optimum = best_score[1]
            best_particle .= particles[best_score[2]]
        end
        
        # figure out which direction each particle needs to go to head towards this best particle
        # need to work on this component
        # and weighting the amount that the velocities change
        direction .= map(x -> sign.(best_particle .- x), particles)

        # add this change in direction to the current velocities vector
        velocities .+= direction
        
        # subtract out 1 + the largest direction to slow down the particle a little bit
        map!(x -> trunc.(Int64, (x ./ (maximum(abs.(x))) .+ sign.(x))), velocities, velocities)
        
        # save the new locations we've visited
        map(x -> push!(visited, x), particles)

        # update the particle positions based on the velocities
        particles .+= velocities

        iter += 1
    end 
    # print(wiggled)
    return(global_optimum, best_particle)
end

# Test the implementation

In [ ]:
Random.seed!(100)
a = TestAlgorithm(100, 1000, 2500)

In [ ]:
BenchmarkTools.@btime(TestAlgorithm(100, 1000, 500))

In [ ]:
Profile.@profile TestAlgorithm(100, 1000, 500)

In [ ]:
Profile.print()

In [ ]:
a = Vector{Int}(undef, 1)
a[1] = 1